# Farmers Using Dual Decomposition

This note has been created by using Julia 1.0.1.

## Required packages

In [1]:
using JuMP, Dsp

┌ Info: Precompiling Dsp [3f8b1f33-babd-5f87-a419-0f6725e165cf]
└ @ Base loading.jl:1189
┌ Warning: Package Dsp does not have MPI in its dependencies:
│ - If you have Dsp checked out for development and have
│   added MPI as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with Dsp
│ Loading MPI into Dsp from project dependency, future warnings for Dsp are suppressed.
└ @ nothing nothing:837


## Parmeters

In [2]:
NS = 3;                        # number of scenarios
probability = [1/3, 1/3, 1/3]; # probability

CROPS  = 1:3 # set of crops (wheat, corn and sugar beets, resp.)
PURCH  = 1:2 # set of crops to purchase (wheat and corn, resp.)
SELL   = 1:4 # set of crops to sell (wheat, corn, sugar beets under 6K and those over 6K)

Cost     = [150 230 260]   # cost of planting crops
Budget   = 500             # budget capacity
Purchase = [238 210];      # purchase price
Sell     = [170 150 36 10] # selling price
Yield    = [3.0 3.6 24.0;
            2.5 3.0 20.0;
            2.0 2.4 16.0]
Minreq   = [200 240 0]     # minimum crop requirement

1×3 Array{Int64,2}:
 200  240  0

## JuMP Model

In [3]:
m = Model(NS)

@variable(m, 0 <= x[i=CROPS] <= 500, Int)
@objective(m, Min, sum(Cost[i] * x[i] for i=CROPS))
@constraint(m, const_budget, sum(x[i] for i=CROPS) <= Budget)

for s in 1:NS
    blk = Model(m, s, probability[s])

    @variable(blk, y[j=PURCH] >= 0)
    @variable(blk, w[k=SELL] >= 0)

    @objective(blk, Min, sum(Purchase[j] * y[j] for j=PURCH) - sum(Sell[k] * w[k] for k=SELL))

    @constraint(blk, const_minreq[j=PURCH], Yield[s,j] * x[j] + y[j] - w[j] >= Minreq[j])
    @constraint(blk, const_minreq_beets, Yield[s,3] * x[3] - w[3] - w[4] >= Minreq[3])
    @constraint(blk, const_aux, w[3] <= 6000)
end

### Query the master problem

In [4]:
print(m)

Min 150 x[1] + 230 x[2] + 260 x[3]
Subject to
 x[1] + x[2] + x[3] ≤ 500
 0 ≤ x[i] ≤ 500, integer, ∀ i ∈ {1,2,3}


### Query a subproblem

In [5]:
print(m.ext[:DspBlocks].children[1])

Min 238 y[1] + 210 y[2] - 170 w[1] - 150 w[2] - 36 w[3] - 10 w[4]
Subject to
 3 x[1] + y[1] - w[1] ≥ 200
 3.6 x[2] + y[2] - w[2] ≥ 240
 24 x[3] - w[3] - w[4] ≥ 0
 w[3] ≤ 6000
 y[j] ≥ 0 ∀ j ∈ {1,2}
 w[k] ≥ 0 ∀ k ∈ {1,2,3,4}


## Dual Decomposition Solution

In [6]:
solve(m)


DUAL DECOMPOSITION ITERATION INFORMATION:
* master   = objective function value of the master problem.
* primobj  = best primal objective function value.
* dualobj  = best dual objective function value.
* a.gap() = Approximate gap between master and dualobj.
* d.gap() = Duality gap between primobj and dualobj.
* times    = wall clock time in seconds.

  iter         master        primobj        dualobj  a.gap(%)  d.gap(%)    time
 B   0  -1.153737e+05  -1.077010e+05  -1.154000e+05      0.02      7.15     0.0
 D   1  -1.153211e+05  -1.077010e+05  -1.153737e+05      0.05      7.12     0.0
 D   2  -1.152159e+05  -1.077010e+05  -1.153211e+05      0.09      7.08     0.0
 D   3  -1.150055e+05  -1.077010e+05  -1.152159e+05      0.18      6.98     0.0
 D   4  -1.145847e+05  -1.077010e+05  -1.150055e+05      0.37      6.78     0.0
 D   5  -1.137431e+05  -1.077010e+05  -1.145847e+05      0.74      6.39     0.0
 D   6  -1.120599e+05  -1.077010e+05  -1.137431e+05      1.50      5.61     0.0
 D   

:Optimal

## Primal and Dual Bounds

There is a duality gap, because the dual decompositino does not guarantee to find a global optimum. The primal bound is obtained by a simple heuristic.

In [7]:
(getprimobjval(), getdualobjval())

(-107700.99999999999, -108390.0)

## Display Dual Variable Values

In [8]:
getdualvalue()

9-element Array{Float64,1}:
  46.53338303892843  
  29.866716372262033 
 -57.1332836277383   
 -12.024044703063884 
 -23.35737803639756  
  49.642621963602835 
 -34.50933833586454  
  -6.509338335864472 
   7.4906616641354695